In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [7]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a') # max
    targetQs = rewards + (gamma * aQs2_logits)
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
    eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min Q
    aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max Q
    eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # minimize Q
    aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # maximize Q
    return actions_logits, aloss, eloss, aloss2

In [8]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [9]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [10]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [11]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [12]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [13]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [14]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:20.0000 R:20.0000 rate:0.0400 aloss:0.7096 eloss:3.5596 aloss2:2.3476 exploreP:0.9980
Episode:1 meanR:20.5000 R:21.0000 rate:0.0420 aloss:0.6977 eloss:3.6005 aloss2:2.3310 exploreP:0.9959
Episode:2 meanR:18.0000 R:13.0000 rate:0.0260 aloss:0.7075 eloss:3.5387 aloss2:2.3253 exploreP:0.9947
Episode:3 meanR:20.0000 R:26.0000 rate:0.0520 aloss:0.7058 eloss:3.5654 aloss2:2.3059 exploreP:0.9921
Episode:4 meanR:19.6000 R:18.0000 rate:0.0360 aloss:0.7132 eloss:3.5379 aloss2:2.2931 exploreP:0.9903
Episode:5 meanR:23.0000 R:40.0000 rate:0.0800 aloss:0.7033 eloss:3.5678 aloss2:2.2756 exploreP:0.9864
Episode:6 meanR:21.4286 R:12.0000 rate:0.0240 aloss:0.6987 eloss:3.5159 aloss2:2.2951 exploreP:0.9853
Episode:7 meanR:20.7500 R:16.0000 rate:0.0320 aloss:0.7110 eloss:3.5731 aloss2:2.2345 exploreP:0.9837
Episode:8 meanR:21.0000 R:23.0000 rate:0.0460 aloss:0.6993 eloss:3.4712 aloss2:2.2814 exploreP:0.9815
Episode:9 meanR:23.1000 R:42.0000 rate:0.0840 aloss:0.6983 eloss:3.4844 aloss2:2.2

Episode:80 meanR:24.9630 R:20.0000 rate:0.0400 aloss:0.6895 eloss:3.1338 aloss2:2.2443 exploreP:0.8188
Episode:81 meanR:24.8902 R:19.0000 rate:0.0380 aloss:0.6872 eloss:3.1618 aloss2:2.1847 exploreP:0.8172
Episode:82 meanR:24.7831 R:16.0000 rate:0.0320 aloss:0.6860 eloss:3.1555 aloss2:2.2287 exploreP:0.8159
Episode:83 meanR:24.6548 R:14.0000 rate:0.0280 aloss:0.6918 eloss:3.1356 aloss2:2.2118 exploreP:0.8148
Episode:84 meanR:24.6000 R:20.0000 rate:0.0400 aloss:0.6902 eloss:3.1273 aloss2:2.2454 exploreP:0.8132
Episode:85 meanR:24.7093 R:34.0000 rate:0.0680 aloss:0.6888 eloss:3.1194 aloss2:2.2690 exploreP:0.8105
Episode:86 meanR:24.7241 R:26.0000 rate:0.0520 aloss:0.6851 eloss:3.1317 aloss2:2.2572 exploreP:0.8084
Episode:87 meanR:24.5909 R:13.0000 rate:0.0260 aloss:0.6881 eloss:3.0986 aloss2:2.2683 exploreP:0.8074
Episode:88 meanR:24.5056 R:17.0000 rate:0.0340 aloss:0.6878 eloss:3.1345 aloss2:2.2133 exploreP:0.8060
Episode:89 meanR:24.4778 R:22.0000 rate:0.0440 aloss:0.6898 eloss:3.1070 

Episode:159 meanR:33.5200 R:33.0000 rate:0.0660 aloss:0.6719 eloss:3.2589 aloss2:2.2463 exploreP:0.6284
Episode:160 meanR:34.1000 R:69.0000 rate:0.1380 aloss:0.6813 eloss:3.2871 aloss2:2.2261 exploreP:0.6242
Episode:161 meanR:34.6900 R:75.0000 rate:0.1500 aloss:0.6923 eloss:3.3132 aloss2:2.2196 exploreP:0.6196
Episode:162 meanR:35.0100 R:71.0000 rate:0.1420 aloss:0.7153 eloss:3.3293 aloss2:2.2147 exploreP:0.6153
Episode:163 meanR:35.7400 R:113.0000 rate:0.2260 aloss:0.7146 eloss:3.3201 aloss2:2.2283 exploreP:0.6085
Episode:164 meanR:35.4800 R:38.0000 rate:0.0760 aloss:0.6834 eloss:3.2943 aloss2:2.2362 exploreP:0.6062
Episode:165 meanR:36.7400 R:153.0000 rate:0.3060 aloss:0.7059 eloss:3.3109 aloss2:2.2399 exploreP:0.5972
Episode:166 meanR:36.9500 R:80.0000 rate:0.1600 aloss:0.7038 eloss:3.2894 aloss2:2.2598 exploreP:0.5925
Episode:167 meanR:36.7000 R:16.0000 rate:0.0320 aloss:0.7268 eloss:3.3257 aloss2:2.2165 exploreP:0.5916
Episode:168 meanR:37.3900 R:84.0000 rate:0.1680 aloss:0.7044 e

Episode:238 meanR:93.1700 R:99.0000 rate:0.1980 aloss:0.7999 eloss:3.3360 aloss2:2.2449 exploreP:0.2786
Episode:239 meanR:92.9600 R:26.0000 rate:0.0520 aloss:0.7421 eloss:3.3367 aloss2:2.2634 exploreP:0.2779
Episode:240 meanR:93.2500 R:128.0000 rate:0.2560 aloss:0.7562 eloss:3.3534 aloss2:2.2583 exploreP:0.2745
Episode:241 meanR:92.9700 R:37.0000 rate:0.0740 aloss:0.7547 eloss:3.3451 aloss2:2.2592 exploreP:0.2735
Episode:242 meanR:93.9900 R:130.0000 rate:0.2600 aloss:0.7576 eloss:3.3679 aloss2:2.2556 exploreP:0.2701
Episode:243 meanR:93.9500 R:30.0000 rate:0.0600 aloss:0.7360 eloss:3.3648 aloss2:2.2667 exploreP:0.2693
Episode:244 meanR:94.4600 R:108.0000 rate:0.2160 aloss:0.7332 eloss:3.3426 aloss2:2.2779 exploreP:0.2665
Episode:245 meanR:95.6500 R:146.0000 rate:0.2920 aloss:0.7714 eloss:3.3442 aloss2:2.2602 exploreP:0.2628
Episode:246 meanR:96.8900 R:139.0000 rate:0.2780 aloss:0.7693 eloss:3.3451 aloss2:2.2843 exploreP:0.2593
Episode:247 meanR:97.4400 R:144.0000 rate:0.2880 aloss:0.76

Episode:317 meanR:71.3900 R:17.0000 rate:0.0340 aloss:0.7330 eloss:3.3287 aloss2:2.2728 exploreP:0.1785
Episode:318 meanR:70.7200 R:47.0000 rate:0.0940 aloss:0.7494 eloss:3.3411 aloss2:2.2738 exploreP:0.1777
Episode:319 meanR:69.3000 R:24.0000 rate:0.0480 aloss:0.7297 eloss:3.3940 aloss2:2.2390 exploreP:0.1773
Episode:320 meanR:68.1500 R:15.0000 rate:0.0300 aloss:0.7315 eloss:3.3322 aloss2:2.2785 exploreP:0.1771
Episode:321 meanR:66.3600 R:21.0000 rate:0.0420 aloss:0.7223 eloss:3.3531 aloss2:2.2560 exploreP:0.1767
Episode:322 meanR:64.8600 R:28.0000 rate:0.0560 aloss:0.7317 eloss:3.3458 aloss2:2.2638 exploreP:0.1763
Episode:323 meanR:63.5300 R:33.0000 rate:0.0660 aloss:0.7665 eloss:3.3255 aloss2:2.2913 exploreP:0.1757
Episode:324 meanR:63.4100 R:23.0000 rate:0.0460 aloss:0.7489 eloss:3.3236 aloss2:2.2820 exploreP:0.1753
Episode:325 meanR:63.3000 R:31.0000 rate:0.0620 aloss:0.7725 eloss:3.3409 aloss2:2.2725 exploreP:0.1748
Episode:326 meanR:61.8600 R:29.0000 rate:0.0580 aloss:0.7444 elo

...

Episode:2724 meanR:125.5000 R:152.0000 rate:0.3040 aloss:0.2345 eloss:2.9352 aloss2:2.7517 exploreP:0.0100
Episode:2725 meanR:125.5000 R:110.0000 rate:0.2200 aloss:0.2191 eloss:2.9345 aloss2:2.7806 exploreP:0.0100
Episode:2726 meanR:124.5500 R:115.0000 rate:0.2300 aloss:0.2257 eloss:2.9047 aloss2:2.7829 exploreP:0.0100
Episode:2727 meanR:124.3900 R:143.0000 rate:0.2860 aloss:0.2335 eloss:2.9342 aloss2:2.7858 exploreP:0.0100
Episode:2728 meanR:124.5000 R:153.0000 rate:0.3060 aloss:0.2397 eloss:2.9572 aloss2:2.7565 exploreP:0.0100
Episode:2729 meanR:124.6300 R:142.0000 rate:0.2840 aloss:0.2303 eloss:2.9683 aloss2:2.7375 exploreP:0.0100
Episode:2730 meanR:124.7900 R:135.0000 rate:0.2700 aloss:0.2519 eloss:2.9652 aloss2:2.7755 exploreP:0.0100
Episode:2731 meanR:124.7800 R:142.0000 rate:0.2840 aloss:0.2586 eloss:2.9625 aloss2:2.7436 exploreP:0.0100
Episode:2732 meanR:125.0400 R:155.0000 rate:0.3100 aloss:0.2380 eloss:2.9471 aloss2:2.7419 exploreP:0.0100
Episode:2733 meanR:125.0600 R:100.000

Episode:2801 meanR:126.7200 R:108.0000 rate:0.2160 aloss:0.3871 eloss:2.9464 aloss2:3.1042 exploreP:0.0100
Episode:2802 meanR:126.6700 R:103.0000 rate:0.2060 aloss:0.3325 eloss:2.9781 aloss2:2.7765 exploreP:0.0100
Episode:2803 meanR:126.7300 R:115.0000 rate:0.2300 aloss:0.3558 eloss:3.0231 aloss2:2.7825 exploreP:0.0100
Episode:2804 meanR:127.3900 R:165.0000 rate:0.3300 aloss:0.3246 eloss:3.3106 aloss2:2.7441 exploreP:0.0100
Episode:2805 meanR:127.5900 R:130.0000 rate:0.2600 aloss:0.2993 eloss:3.8593 aloss2:2.9086 exploreP:0.0100
Episode:2806 meanR:127.3500 R:105.0000 rate:0.2100 aloss:0.3122 eloss:3.7508 aloss2:3.0954 exploreP:0.0100
Episode:2807 meanR:126.6600 R:56.0000 rate:0.1120 aloss:0.3399 eloss:3.5268 aloss2:2.9585 exploreP:0.0100
Episode:2808 meanR:126.7500 R:108.0000 rate:0.2160 aloss:0.3953 eloss:3.4411 aloss2:3.0377 exploreP:0.0100
Episode:2809 meanR:126.1200 R:57.0000 rate:0.1140 aloss:0.3902 eloss:3.3365 aloss2:3.0444 exploreP:0.0100
Episode:2810 meanR:125.6600 R:64.0000 r

Episode:2879 meanR:73.0100 R:45.0000 rate:0.0900 aloss:0.3700 eloss:2.0511 aloss2:4.5292 exploreP:0.0100
Episode:2880 meanR:72.1700 R:31.0000 rate:0.0620 aloss:0.3994 eloss:1.9303 aloss2:4.8736 exploreP:0.0100
Episode:2881 meanR:71.1500 R:29.0000 rate:0.0580 aloss:0.4860 eloss:2.1152 aloss2:4.7090 exploreP:0.0100
Episode:2882 meanR:70.8600 R:47.0000 rate:0.0940 aloss:0.3743 eloss:1.9430 aloss2:4.7411 exploreP:0.0100
Episode:2883 meanR:70.4200 R:43.0000 rate:0.0860 aloss:0.3977 eloss:2.0858 aloss2:4.6545 exploreP:0.0100
Episode:2884 meanR:69.8700 R:35.0000 rate:0.0700 aloss:0.3564 eloss:1.3608 aloss2:5.6772 exploreP:0.0100
Episode:2885 meanR:68.9700 R:29.0000 rate:0.0580 aloss:0.4357 eloss:1.4712 aloss2:5.4552 exploreP:0.0100
Episode:2886 meanR:67.8500 R:32.0000 rate:0.0640 aloss:0.4247 eloss:2.1096 aloss2:4.5710 exploreP:0.0100
Episode:2887 meanR:66.4400 R:33.0000 rate:0.0660 aloss:0.3937 eloss:1.4393 aloss2:5.3334 exploreP:0.0100
Episode:2888 meanR:65.6700 R:37.0000 rate:0.0740 aloss:

Episode:2958 meanR:44.4000 R:50.0000 rate:0.1000 aloss:0.4505 eloss:0.7872 aloss2:5.1795 exploreP:0.0100
Episode:2959 meanR:44.4600 R:37.0000 rate:0.0740 aloss:0.4018 eloss:0.8573 aloss2:5.1315 exploreP:0.0100
Episode:2960 meanR:43.9400 R:50.0000 rate:0.1000 aloss:0.4233 eloss:1.1020 aloss2:4.8030 exploreP:0.0100
Episode:2961 meanR:43.8300 R:31.0000 rate:0.0620 aloss:0.3861 eloss:1.3509 aloss2:4.5846 exploreP:0.0100
Episode:2962 meanR:43.1400 R:35.0000 rate:0.0700 aloss:0.3929 eloss:0.8569 aloss2:5.0422 exploreP:0.0100
Episode:2963 meanR:43.1000 R:35.0000 rate:0.0700 aloss:0.4023 eloss:0.5947 aloss2:5.3389 exploreP:0.0100
Episode:2964 meanR:42.3900 R:43.0000 rate:0.0860 aloss:0.4425 eloss:0.8431 aloss2:5.0947 exploreP:0.0100
Episode:2965 meanR:42.2900 R:33.0000 rate:0.0660 aloss:0.3881 eloss:0.7243 aloss2:5.2126 exploreP:0.0100
Episode:2966 meanR:41.4800 R:37.0000 rate:0.0740 aloss:0.3693 eloss:1.1767 aloss2:4.7200 exploreP:0.0100
Episode:2967 meanR:41.5700 R:40.0000 rate:0.0800 aloss:

Episode:3037 meanR:43.4000 R:39.0000 rate:0.0780 aloss:0.3126 eloss:2.3584 aloss2:3.3697 exploreP:0.0100
Episode:3038 meanR:43.3200 R:27.0000 rate:0.0540 aloss:0.3100 eloss:2.2228 aloss2:3.5304 exploreP:0.0100
Episode:3039 meanR:44.1800 R:123.0000 rate:0.2460 aloss:0.3253 eloss:2.3248 aloss2:3.4127 exploreP:0.0100
Episode:3040 meanR:44.9600 R:113.0000 rate:0.2260 aloss:0.2833 eloss:2.3648 aloss2:3.3818 exploreP:0.0100
Episode:3041 meanR:45.6600 R:107.0000 rate:0.2140 aloss:0.2849 eloss:2.5500 aloss2:3.2106 exploreP:0.0100
Episode:3042 meanR:46.5400 R:131.0000 rate:0.2620 aloss:0.2721 eloss:2.6524 aloss2:3.0865 exploreP:0.0100
Episode:3043 meanR:47.3400 R:125.0000 rate:0.2500 aloss:0.2875 eloss:2.6678 aloss2:3.0994 exploreP:0.0100
Episode:3044 meanR:48.0500 R:106.0000 rate:0.2120 aloss:0.2561 eloss:2.7923 aloss2:2.9723 exploreP:0.0100
Episode:3045 meanR:48.6200 R:102.0000 rate:0.2040 aloss:0.2596 eloss:2.9121 aloss2:2.8349 exploreP:0.0100
Episode:3046 meanR:49.5100 R:130.0000 rate:0.260

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
